In [6]:
import numpy as np

In [7]:
class NeuronCell(object):
    def __init__(self, ID):
        # 自身ID
        self.id = ID
        # 单数据最大值
        self.dataMax = 100
        # 数据 id: input data
        self.data = {
            'in': {}, 
            'out': {}
        }
        # 权值 id: weight
        self.weights = {
            'in': {}, 
            'out': {}
        }
        # 聚合数据大小
        self.totalData = 0
        # 激发阙值
        self.threshold = 15
        # 最大输入输出端口
        self.portMax = {
            'in': 1000, 
            'out': 1000
        }
        # 连接数
        self.connected = {
            'in': 0, 
            'out': 0
        }
        # 目标神经元
        self.targetNeurons = None
        
    def setMaxConnection(self, maxIn=1000, maxOut=1000):
        self.portMax['in'], self.portMax['out'] = maxIn, maxOut
    
    # conn = True or False, port = 'in' or 'out'
    def conn(self, conn, port, ID):
        if self.connected[port] >= self.portMax[port]:
            # 连接失败
            return False
        if conn:
            self.data[port][ID] = 0
            self.weights[port][ID] = 0.5
            self.sortByID(port)
            self.connected[port] += 1
        else:
            del self.data[port][ID]
            del self.weights[port][ID]
            self.sortByID(port)
            self.connected[port] -= 1
        self.targetNeurons = set(self.data['out'].keys())
        # 连接成功
        return True
    
    def inputData(self, data, sourceID):
        self.data['in'][sourceID] = data
    
    def prepareForFire(self):
        inputs = np.array(list(self.data['in'].values()), dtype=np.float16)
        weightsIn = np.array(list(self.weights['in'].values()), dtype=np.float16)
        self.totalData = np.sum(inputs * weightsIn) / len(self.data['in'])
        for Id in self.data['in'].keys():
            self.data['in'][Id] = 0
    
    def setTotalData(self, data):
        self.totalData = data
        
    def fire(self):
        if self.totalData > self.threshold:
            weightsOut = np.array(self.weights['out'], dtype=np.float16)
            dataOut = self.leakyRelu(weightsOut * self.totalData).tolist()
            IDs = list(self.data['out'].keys())
            self.data['out'] = dict(zip(IDs, dataOut))
            self.totalData = 0
            # 返回激发成功
            return True
        # 激发失败
        return False
            
    def transToTarget(self, neurons):
        for Id, data in self.data['out']:
            neurons[Id].inputData(data, self.id)
            self.data['out'][Id] = 0
        return self.targetNeurons
    
    def sortById(self, port):
        if port == 'all':
            self.data['in'] = dict(sorted(self.data['in'].items()))
            self.data['out'] = dict(sorted(self.data['out'].items()))
            self.weights['in'] = dict(sorted(self.weights['in'].items()))
            self.weights['out'] = dict(sorted(self.weights['out'].items()))
        else:
            self.data[port] = dict(sorted(self.data[port].items()))
            self.weights[port] = dict(sorted(self.weights[port].items()))
    
    def leakyRelu(self, x, p=0.01):
        return np.maximum(x, x*p)
    
    def updateWeight(self, port, neuronID, times):
        self.weights[port][neuronID] *= times
        self.weights[port][neuronID] = min(self.weights[port][neuronID], 1)
        
    # stdp学习方法，两个神经元放电时间 <= 5ms,则连接权重增加70%, 否则减少20%
    def stdp(self, fire, totalNeurons):
        # 1.7^(1/2) = 1.3   0.8^(1/2) = 0.89
        weightTimes = {
            'in': (1.25, 0.95), 
            'out': (1.4, 0.85)
        }
        index = 0 if fire else 1
        for port, weights in self.weights.items():
            for ID, weight in weights.items():
                # 如果本神经元fire为True,则增加权重，否则减少
                self.updateWeight(port, ID, weightTimes[port][index])
                if port == 'in':
                    totalNeurons[ID].updateWeight('out', self.id, weightTimes['out'][index])
    
    def getParams(self):
        params = {
            'weights': self.weights, 
            'threshold': self.threshold, 
            'connected': self.connected
        }
        return params

In [3]:
class Brain(object):
    def __init__(self):
        # 神经元数量
        self.neuronNum = 0
        # 邻接矩阵，行 是否连接 列
        # 例：self.Adjaceny['from'][1][2] = 1, 代表id:1 from id:2 True
        #     self.Adjaceny['to'][1][2] = 1, 代表id:1 to id:2 True
        self.Adjacency = {
            'from': None, 
            'to': None
        }
        # 所有神经元, id: neuron
        self.totalNeurons = {}
        # 神经元分区
        self.sections = {
            'input':dict(), 
            'think':dict(), 
            'output':dict()
        }
        # 准备发射神经元ID(梯队)
        self.readyNeurons = {
            0: set(), 
            1: set()
        }
    
    # 增加神经元
    def createNeuron(self, number):
        newNum = self.neuronNum + number
        for ID in range(self.neuronNum, newNum):
            self.totalNeurons[ID] = NeuronCell(ID)
        temp = self.Adjaceny
        self.Adjacency = {
            'from': np.zeros((newNum, newNum)), 
            'to': np.zeros((newNum, newNum))
        }
        self.Adjaceny['from'][0:self.neuronNum][0:self.neuronNum] = temp['from']
        self.Adjaceny['to'][0:self.neuronNum][0:self.neuronNum] = temp['to']
        self.neuronNum = newNum
        
    
    # 删除神经元
    def deleteNeuron(self, IDs):
        for ID in IDs:
            try:
                del self.totalNeurons[ID]
            except:
                continue
                
    # 连接神经元
    def connectNeuron(self, sourceID, targetID):
        source.conn(True, 'out', targetID)
        target.conn(True, 'in', sourceID)
        self.Adjacency['from'][targetID][sourceId] = 1
        self.Adjacency['to'][sourceID][targetId] = 1
    
    # 断开连接
    def disconnect(self, sourceID, targetID):
        source.conn(False, 'out', targetID)
        target.conn(False, 'in', sourceID)
        self.Adjacency['from'][targetID][sourceId] = 0
        self.Adjacency['to'][sourceID][targetId] = 0
    
    # 分配初始神经元
    def assignNeuron(self, section, index, neuronIds):
        for ID in neuronIds:
            self.sections[section][index].add(ID)
    
    # 全连接指定神经元群
    def fullConn(self, neuronIDs):
        for ID1 in neurons:
            for ID2 in neurons:
                self.connectNeuron(ID1, ID2)
                if ID1 != ID2:
                    self.connectNeuron(ID2, ID1)
    
    # 单向全连接
    def oneWayFullConn(self, source, target):
        for ID_From in source:
            for ID_To in target:
                self.connectNeuron(ID_From, ID_To)
    
    # 增加群组
    def addGroup(self, section, ID, neuronIDs):
        self.sections[section][ID] = set(neuronIDs)
        
    # 初始化神经元连接
    def initialConn(self, **kwargs):
        # MAP 0
        
        # CREATE SECTIONS, ASSIGN NEURONS
        # input 0: vision, 1: hearing
        self.addGroup('input', 0, kwargs['inputVision'])
        self.addGroup('input', 1, kwargs['inputHearing'])
        # output 0: draw(vision), 1: sound(hearing)
        self.addGroup('input', 0, kwargs['outputVision'])
        self.addGroup('input', 1, kwargs['outputHearing'])
        # think 0:Brain0
        # 'receiving0': part neurons of Brain0 that receiving the vision input and spread to whole think network
        # 'receiving1': part neurons of Brain0 that receiving the hearing input and spread to whole think network
        # 'send0': part neurons of Brain0 that send the vision data and spread to drawing output
        # 'send1': part neurons of Brain0 that send the hearing data and spread to sound output
        self.addGroup('think', 0, kwargs['think0'])
        self.addGroup('think', 'receiving0', kwargs['think0'][0:len(kwargs['inputVision'])])
        self.addGroup('think', 'receiving1', kwargs['think0'][0:len(kwargs['inputHearing'])])
        self.addGroup('think', 'send0', kwargs['think0'][0:len(kwargs['outputVision'])])
        self.addGroup('think', 'send1', kwargs['think0'][0:len(kwargs['outputHearing'])])
        
        # CONNECT
        # Input ==> Brain0, One way
        self.oneWayFullConn(self.sections['input'][0], self.sections['think']['receiving0'])
        self.oneWayFullConn(self.sections['input'][1], self.sections['think']['receiving1'])
        # full connect brain0
        self.fullConn(self.sections['think'][0])
        # Brain0 ==> Output, One way
        self.oneWayFullConn(self.sections['think']['send0'], self.sections['output'][0])
        self.oneWayFullConn(self.sections['think']['send1'], self.sections['output'][1])
        
    
    # 想
    def think(self, steps=1000):
        for i in range(steps):
            for id_ready in self.readyNeurons[0]:
                neuron = self.totalNeurons[id_ready]
                neuron.prepareForFire()
                if neuron.fire():
                    self.readyNeurons[1] = self.readyNeurons[1] | neuron.transToTarget(self.totalNerons)
            self.readyNeurons[0] = self.readyNeurons[1]
    
    # 输入数据(数据已加工，(原数据 * 传输数据最大限制)/最大原数据)
    def inputs(self, data, sectID):
        neuronIDs = self.sections['input'][sectID]
        
        if len(data) > len(neuronIDs):
            print('data length more than number of input neurons')
            return False
        
        for index, ID in enumerate(neurons):
            self.totalNeurons[ID].setTotalData(data[index])
            self.readyNeurons[0].add(ID)
        
        return True
    
    # 输出
    def outputs(self, sectID, maxData, minData):
        neuronIDs = self.sections['output'][sectID]
        
        outputs = np.empty(len(neuronIDs))
        
        for i, ID in enumerate(neuronIDs):
            outputs[i] = self.totalNeurons[ID].totalData
            
        outputs[outputs>maxData] = maxData
        outputs[outputs<minData] = minData
        
        return outputs
    
    # 更新权重
    def updateWeights(self):
        pass
    
    # 保存
    def save(self):
        params = {
            'neuronNum': self.neuronNum, 
            'Adjacency': self.Adjacency, 
            'sections': self.sections, 
            'neuronParams': {}
        }
        for ID, neuron in self.totalNeurons:
            params['neuronParams'][ID] = self.totalNeurons[ID].getParams()
            